In [1]:
import pandas as pd
import numpy as np
import math
import os

In [2]:
# create category table
# economy has 3 level categories
level1 = ["耕地面积 Cultivated Area", "粮食总产量 Total Grain Output", 
          "人均收入 Per Capita Income", "人均居住面积 Per Capita Living Space",
          "总产值 Gross Output Value", "集体经济收入 Collective Economic Income",
          "电价 Electricity Price", "用电量 Electricity Consumption", 
          "水价 Water Price", "用水量 Water Consumption"]

level2 = ['总 Total', 
          '第一产业 Raw Materials','第二产业 Secondary Industry','第三产业 Service',
          '种植业 Agriculture','林业 Forestry', '牧业 Animal Husbandry', '副业 Miscellaneous', '渔业 Fishery',  
         'General', '生活 Household','农业 Agricultural', '工业 Industrial', '商业 Commercial']

level3 = ['粮食 Grain', '种植业', '水果 Fruit', '蔬菜 Vegetables',
         '运输业 Shipping', '仓储业 Warehouse', '建筑业 Construction',
         '商饮 Grocery', '服务业 Hospitality and Service',
         '全村 village', '每户 per household', '每人 per person']

# level2 categories under 总产值，集体经济收入
level2_Gross_and_Collective = ['总 Total','第一产业 Raw Materials','第二产业 Secondary Industry','第三产业 Service',
           '种植业 Agriculture','林业 Forestry', '牧业 Animal Husbandry', '副业 Miscellaneous', '渔业 Fishery']
# level2 categories under 水电
level2_Water_Electricity = ['General', '生活 Household','农业 Agricultural', '工业 Industrial', '商业 Commercial']

# level3 categories under 种植业
level3_Agri = ['粮食 Grain', '种植业', '水果 Fruit', '蔬菜 Vegetables']
# level3 categories under 副业
level3_Misc = ['运输业 Shipping', '仓储业 Warehouse', '建筑业 Construction']
# level3 categories under 服务业
level3_Serviec = ['商饮 Grocery', '服务业 Hospitality and Service']
# level3 categories under 生活（电价，用电量下的生活类）
level3_Household = ['全村 village', '每户 per household', '每人 per person']

# print(list(df['Category'].astype('category').unique()))
# print(list(df['Division1'].astype('category').unique()))
# print(list(df['Division2'].astype('category').unique()))
# print(list(df['Subdivision - Agri.'].astype('category').unique()))
# print(list(df['Subdivision - Misc.'].astype('category').unique()))
# print(list(df['Subdivision - Service'].astype('category').unique()))
# print(list(df['Subdivision - Household'].astype('category').unique()))

def add_to_category_id(category_id, name, count):
    if name not in category_id:
        category_id[name] = []
        category_id[name].append(count)
    else:
        category_id[name].append(count)
    
    
# prepare for category_df
dic_category_df = {'id':[], 'name':[], 'parentId':[]}
category_id = {}
# count for id
count = 1

# create level1 categories
for category in level1:
    dic_category_df['id'].append(count)
    dic_category_df['name'].append(category)
    dic_category_df['parentId'].append('NULL')
    add_to_category_id(category_id, category, count)
    count = count + 1

# create level2 categories under "总产值" , "集体经济收入" 
for parent in ["总产值 Gross Output Value", "集体经济收入 Collective Economic Income"]:
    for parentId in category_id[parent]:
        for category in level2_Gross_and_Collective:
            dic_category_df['id'].append(count)
            dic_category_df['name'].append(category)
            dic_category_df['parentId'].append(parentId)
            add_to_category_id(category_id, category, count)
            count = count + 1
        
# create level2 categories under 电价 , 用电量 , 水价  用水量 
for parent in ["电价 Electricity Price", "用电量 Electricity Consumption", 
          "水价 Water Price", "用水量 Water Consumption"]:
    for parentId in category_id[parent]:
        for category in level2_Water_Electricity:
            dic_category_df['id'].append(count)
            dic_category_df['name'].append(category)
            dic_category_df['parentId'].append(parentId)
            add_to_category_id(category_id, category, count)
            count = count + 1
            
# create level3 categories under '种植业 Agriculture'
for parent in ["种植业 Agriculture"]:
    for parentId in category_id[parent]:
        for category in level3_Agri:
            dic_category_df['id'].append(count)
            dic_category_df['name'].append(category)
            dic_category_df['parentId'].append(parentId)
            add_to_category_id(category_id, category, count)
            count = count + 1

# create level3 categories under '副业 Miscellaneous'
for parent in ["副业 Miscellaneous"]:
    for parentId in category_id[parent]:
        for category in level3_Misc:
            dic_category_df['id'].append(count)
            dic_category_df['name'].append(category)
            dic_category_df['parentId'].append(parentId)
            add_to_category_id(category_id, category, count)
            count = count + 1

# create level3 categories under '第三产业 Service'
for parent in ["第三产业 Service"]:
    for parentId in category_id[parent]:
        for category in level3_Serviec:
            dic_category_df['id'].append(count)
            dic_category_df['name'].append(category)
            dic_category_df['parentId'].append(parentId)
            add_to_category_id(category_id, category, count)
            count = count + 1

# create level3 categories under '生活 Household' of 用电量 and 电价
for parent in ["生活 Household"]:
    for parentId in [30,35]:
        for category in level3_Household:
            dic_category_df['id'].append(count)
            dic_category_df['name'].append(category)
            dic_category_df['parentId'].append(parentId)
            add_to_category_id(category_id, category, count)
            count = count + 1

# create category_df
category_df = pd.DataFrame(columns = ['id', 'name', 'parentId'])
for columns in category_df.columns:
    category_df[columns] = dic_category_df[columns]

# using category_df create category_view
level1_df = category_df[category_df['parentId'] == "NULL"]
category_all = pd.merge(level1_df, category_df, how='left', left_on='id', right_on='parentId')
category_all = pd.merge(category_all, category_df, how='left', left_on='id_y', right_on='parentId')
category_all.columns = ['level1_id', 'level1_name', 'level1_parentId', 
                        'level2_id', 'level2_name', 'level2_parentId', 
                        'level3_id', 'level3_name', 'level3_parentId']
category_all = category_all.where(category_all.notnull(), "NULL")
# category_all

* level1 -- 'Category'
* level2 -- 'Division1', 'Division2'
* level3 -- 'Subdivision - Agri.', 'Subdivision - Misc.', 'Subdivision - Service', 'Subdivision - Household'


In [3]:
path = os.path.abspath(os.getcwd())
# path = "/Users/nqzeng/Desktop/My_WAY/找工作/ccvg/Data"
print(path)

/Users/nqzeng/Desktop/My_WAY/找工作/ccvg/Data


In [4]:
# 1. read yearly and range data
df = pd.read_csv(path + "/Data4/Economy - Yearly.csv")
df = df.dropna(axis = 0, how = 'all')
df.drop_duplicates(inplace = True)

df2 = pd.read_csv(path + "/Data4/Economy - Range.csv")
df2 = df2.dropna(axis = 0, how = 'all')
df2.drop_duplicates(inplace = True)

# 2. create level1, level2, level3 columns
# level1 -- 'Category'
# level2 -- 'Division1', 'Division2'
# level3 -- 'Subdivision - Agri.', 'Subdivision - Misc.', 'Subdivision - Service', 'Subdivision - Household'

# create new columns at df and df2
new_columns = ['level1', 'level2', 'level3']
for column in new_columns:
    df[column] = None
for column in new_columns:
    df2[column] = None
    
# replace NaN in category columns as ""
columns_replace_null = ['Division1', 'Division2','Subdivision - Agri.', 'Subdivision - Misc.', 'Subdivision - Service', 'Subdivision - Household']
for column in columns_replace_null:
    df[column] = df[column].where(df[column].notnull(), "")
    df2[column] = df2[column].where(df2[column].notnull(), "")

# create level1, level2, and level3 columns
df['level1'] = df['Category']
df['level2'] = df['Division1'] + df['Division2']
df['level3'] = df['Subdivision - Agri.'] + df['Subdivision - Misc.'] + df['Subdivision - Service'] + df['Subdivision - Household']

df2['level1'] = df2['Category']
df2['level2'] = df2['Division1'] + df2['Division2']
df2['level3'] = df2['Subdivision - Agri.'] + df2['Subdivision - Misc.'] + df2['Subdivision - Service'] + df2['Subdivision - Household']


# replace "" and NaN as "NULL"
df = df.where(df != "", "NULL")
df = df.where(df.notnull(), "NULL")

df2 = df2.where(df2 != "", "NULL")
df2 = df2.where(df2.notnull(), "NULL")
df.head()

/Users/nqzeng/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,村志代码 Gazetteer Code,村志书名 Gazetteer Title,Category,Division1,Subdivision - Agri.,Subdivision - Misc.,Subdivision - Service,Division2,Subdivision - Household,Unit,...,2013,2014,2015,2016,2017,2018,2019,level1,level2,level3
0,1,太平店村志,耕地面积 Cultivated Area,NULL,NULL,NULL,NULL,NULL,NULL,亩 mu,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,耕地面积 Cultivated Area,NULL,NULL
1,1,太平店村志,粮食总产量 Total Grain Output,NULL,NULL,NULL,NULL,NULL,NULL,吨 tons,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,粮食总产量 Total Grain Output,NULL,NULL
2,1,太平店村志,人均收入 Per Capita Income,NULL,NULL,NULL,NULL,NULL,NULL,元 yuan,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,人均收入 Per Capita Income,NULL,NULL
3,2,叶店村志,总产值 Gross Output Value,总 Total,NULL,NULL,NULL,NULL,NULL,万元 10K yuan,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,总产值 Gross Output Value,总 Total,NULL
4,2,叶店村志,集体经济收入 Collective Economic Income,总 Total,NULL,NULL,NULL,NULL,NULL,万元 10K yuan,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,集体经济收入 Collective Economic Income,总 Total,NULL


In [5]:
# merge yearly df and range df into df_yearly_range
# prepare for df_yearly_range
dic_df_yearly_range = {'gazetteerId':[], 'level1':[], 'level2':[], 'level3':[], 'categoryId':[], 'startYear':[], 'endYear':[], 'data':[], 'unit':[]}
years = [str(i) for i in range(1949,2020)]
level1 = ['Category']
level2 = ['Division1', 'Division2']

for i in range(len(df)):# each row
    for year in years: # 1949 - 2019
        if df.iloc[i][year] != "NULL":
            dic_df_yearly_range['gazetteerId'].append(df.iloc[i]['村志代码 Gazetteer Code'])
            dic_df_yearly_range['level1'].append(df.iloc[i]['level1'])
            dic_df_yearly_range['level2'].append(df.iloc[i]['level2'])
            dic_df_yearly_range['level3'].append(df.iloc[i]['level3'])
            dic_df_yearly_range['categoryId'].append(None)
            dic_df_yearly_range['startYear'].append(int(year))
            dic_df_yearly_range['endYear'].append(int(year))
            dic_df_yearly_range['data'].append(df.iloc[i][year])
            dic_df_yearly_range['unit'].append(df.iloc[i]['Unit'])

for i in range(len(df2)):# each row
    dic_df_yearly_range['gazetteerId'].append(df2.iloc[i]['村志代码 Gazetteer Code'])
    dic_df_yearly_range['level1'].append(df2.iloc[i]['level1'])
    dic_df_yearly_range['level2'].append(df2.iloc[i]['level2'])
    dic_df_yearly_range['level3'].append(df2.iloc[i]['level3'])
    dic_df_yearly_range['categoryId'].append(None)
    dic_df_yearly_range['startYear'].append(df2.iloc[i]['Start Year'])
    dic_df_yearly_range['endYear'].append(df2.iloc[i]['End Year'])
    dic_df_yearly_range['data'].append(df2.iloc[i]['Data'])
    dic_df_yearly_range['unit'].append(df2.iloc[i]['Unit'])

In [6]:
# create df_yearly_range
df_yearly_range = pd.DataFrame(columns = ['gazetteerId', 'level1', 'level2', 'level3', 'categoryId', 'startYear', 'endYear', 'data', 'unit'])
for column in df_yearly_range.columns:
    df_yearly_range[column] = dic_df_yearly_range[column]

# df_yearly_range = df_yearly_range.where(df_yearly_range.notnull(), "NULL")
df_yearly_range.head()


,gazetteerId,level1,level2,level3,categoryId,startYear,endYear,data,unit
0,1,耕地面积 Cultivated Area,NULL,NULL,None,2008,2008,3232,亩 mu
1,1,粮食总产量 Total Grain Output,NULL,NULL,None,2001,2001,2000,吨 tons
2,1,人均收入 Per Capita Income,NULL,NULL,None,2000,2000,2565,元 yuan
3,1,人均收入 Per Capita Income,NULL,NULL,None,2001,2001,2594,元 yuan
4,1,人均收入 Per Capita Income,NULL,NULL,None,2008,2008,3000,元 yuan


In [7]:
categoryIds = []

# update categoryId
    # if level3 is not NULL, record id of level3
    # else if level2 is not NULL, record id of level2
    # else if level1 is not NULL, record id of level1
    # else if level1 is NULL, find a wrong record

for i in range(len(df_yearly_range)):
    
    # three level categories of row i
    level1_name = df_yearly_range.iloc[i]['level1']
    level2_name = df_yearly_range.iloc[i]['level2']
    level3_name = df_yearly_range.iloc[i]['level3']
    
    # id of level1 category
    if level1_name != "NULL":
        category = category_all[category_all['level1_name'] == level1_name]
        categoryId = list(category['level1_id'])
    # id of level2 category
    if level2_name != "NULL":
        category = category[category['level2_name'] == level2_name]
        categoryId = list(category['level2_id'])
    # id of level3 category
    if level3_name != "NULL":
        category = category[category['level3_name'] == level3_name]
        categoryId = list(category['level3_id'])
    
    # categoryId is a list, add the first index
    categoryIds.append(categoryId[0])
    

df_yearly_range['categoryId'] = categoryIds


In [8]:
# create dict "unit_id" stores {unit: id}
unit_id = {}
count = 1

for unit in df_yearly_range['unit'].astype('category').unique():
    if unit not in unit_id:
        unit_id[unit] = count
        count = count + 1

dic_unit_df = {'id':[], 'name':[]}
for unit in unit_id:
    dic_unit_df['id'].append(unit_id[unit])
    dic_unit_df['name'].append(unit)
    
unit_df = pd.DataFrame(columns = ['id', 'name'])  
for attribute in unit_df.columns:
    unit_df[attribute] = dic_unit_df[attribute]

# replace unit with unit id
for unit in unit_id:
    df_yearly_range = df_yearly_range.replace(unit, unit_id[unit])

In [9]:
df_yearly_range.head()

,gazetteerId,level1,level2,level3,categoryId,startYear,endYear,data,unit
0,1,耕地面积 Cultivated Area,NULL,NULL,1.0,2008,2008,3232,1
1,1,粮食总产量 Total Grain Output,NULL,NULL,2.0,2001,2001,2000,2
2,1,人均收入 Per Capita Income,NULL,NULL,3.0,2000,2000,2565,3
3,1,人均收入 Per Capita Income,NULL,NULL,3.0,2001,2001,2594,3
4,1,人均收入 Per Capita Income,NULL,NULL,3.0,2008,2008,3000,3


In [18]:
df_duplicate = df_yearly_range[df_yearly_range['gazetteerId'] == 97]
df_duplicate[df_duplicate['categoryId'] == 20]

,gazetteerId,level1,level2,level3,categoryId,startYear,endYear,data,unit
4409,97,集体经济收入 Collective Economic Income,总 Total,NULL,20.0,1962,1962,0.17,4
4410,97,集体经济收入 Collective Economic Income,总 Total,NULL,20.0,1975,1975,0.25,4
4411,97,集体经济收入 Collective Economic Income,总 Total,NULL,20.0,1981,1981,1.68,4
4412,97,集体经济收入 Collective Economic Income,总 Total,NULL,20.0,1985,1985,2,4
4413,97,集体经济收入 Collective Economic Income,总 Total,NULL,20.0,1990,1990,4.41,4
4414,97,集体经济收入 Collective Economic Income,总 Total,NULL,20.0,1995,1995,6.1,4
4415,97,集体经济收入 Collective Economic Income,总 Total,NULL,20.0,2000,2000,1.91,4
4416,97,集体经济收入 Collective Economic Income,总 Total,NULL,20.0,2005,2005,4,4
4417,97,集体经济收入 Collective Economic Income,总 Total,NULL,20.0,2010,2010,17,4
4418,97,集体经济收入 Collective Economic Income,总 Total,NULL,20.0,2014,2014,69.82,4


In [19]:
economy_df = pd.DataFrame(columns = ['gazetteerId', 'categoryId', 'startYear', 'endYear', 'data', 'unit'])
for column in economy_df.columns:
    economy_df[column] = df_yearly_range[column]

economy_df.to_csv('economy_df2.csv', index = False, na_rep = "NULL")

In [12]:
# category_df.to_csv('economy_category_df2.csv', index = False, na_rep = "NULL")

In [13]:
# unit_df.to_csv('economy_unit_df_df2.csv', index = False, na_rep = "NULL")